In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from scipy.interpolate import interp1d
import heapq

In [3]:
def get_filenames(path, ext):
    X0 = []
    for i in sorted(os.listdir(path)):
        if i.endswith(ext):
            X0.append(os.path.join(path, i))
    return X0
    
fileImages = get_filenames('OCT_DATASET/dataset/PatientIDs128_RPE/val/Images', 'npy')
fileMasks = get_filenames('OCT_DATASET/dataset/PatientIDs128_RPE/val/Masks', 'npy')

pick = 1
# img = Image.open('dataset/128_3C_May/Images/' + filename)
# msk = Image.open('dataset/128_3C_May/Masks/' + filename)
img = np.load(fileImages[pick])
msk = np.load(fileMasks[pick])
msk = (np.array(msk) / 255)

FileNotFoundError: [Errno 2] No such file or directory: 'OCT_DATASET/dataset/PatientIDs128_RPE/val/Images'

In [ ]:
mul = np.multiply(img, msk).astype('float')

plt.figure(figsize=(6,6))
plt.imshow(img, cmap='gray')
plt.axis('off')
plt.show()

plt.figure(figsize=(6,6))
plt.imshow(mul, cmap='gray')
plt.axis('off')
plt.show()

def get_max1_max2(img):
    max1 = []
    max2 = []
    size = 2
    k = 0
    for i in range(size, img.shape[1], size):
        indices = (-img[:, i - size:i].reshape(-1)).argsort()[:2]
        row1 = (int)(indices[0] / size)
        row2 = (int)(indices[1] / size)
        col1 = indices[0] -(row1 * size)
        col2 = indices[1] -(row2 * size)
        temp1 = row1, col1 + k
        temp2 = row2, col2 + k
        k += size
        max1.append(temp1)
        max2.append(temp2)

    max1 = np.array(max1)
    max2 = np.array(max2)
    x1 = max1[:, 1]
    y1 = max1[:, 0]
    x2 = max2[:, 1]
    y2 = max2[:, 0]
    return x1, x2, y1, y2


x1, x2, y1, y2 = get_max1_max2(mul)
f1 = interp1d(x1, y1, kind='linear', fill_value="extrapolate")
f2 = interp1d(x2, y2, kind='linear', fill_value="extrapolate")
xnew = np.linspace(0, mul.shape[1], num=1000)

plt.figure(figsize=(12,5), dpi=96)
plt.plot(xnew, f1(xnew), linewidth=0.5)
plt.plot(xnew, f2(xnew), linewidth=0.5)
plt.scatter(x1, y1, alpha=1, marker='*', s=15)
plt.scatter(x2, y2, alpha=0.5, s=7)
plt.legend(['max1', 'max2'], loc='best')
ax = plt.gca()
ax.invert_yaxis()
plt.xlabel('Image Width (Pixels)', fontsize=14, fontweight='bold')
plt.ylabel('Image Height (Pixels)', fontsize=14, fontweight='bold')
plt.show()

plt.figure(figsize=(14,20), dpi=96)
plt.imshow(mul, cmap='gray')
plt.plot(xnew, f1(xnew), linewidth=0.5, c='green')
plt.plot(xnew, f2(xnew), linewidth=0.5, c='red')
plt.scatter(x1, y1, alpha=1, marker='*', s=10, c='green')
plt.scatter(x2, y2, alpha=0.5, s=10, c='red')
plt.xlim([0, mul.shape[1]])
plt.ylim([70, 90])
ax = plt.gca()
ax.invert_yaxis()
plt.xlabel('Image Width (Pixels)', fontsize=14, fontweight='bold')
plt.ylabel('Image Height (Pixels)', fontsize=14, fontweight='bold')
plt.legend(['max1', 'max2'], loc='best')
plt.title('Image Original', fontsize=22, fontweight='bold')
plt.show()


Total variation

$V(x)=\sum_{n}\lvert{x_{n+1}, x_{n}}\lvert$

Given an input signal $x_{n}$, the goal of total variation denoising is to find an approximation, call it  $y_{n}$, that has smaller total variation than $x_{n}$ but is "close" to $x_{n}$. One measure of closeness is the sum of square errors:

$E(x, y)=\frac{1}{n}\sum_{n}({x_{n} - y_{n}})^2$

So the total-variation denoising problem amounts to minimizing the following discrete functional over the signal $y_{n}$:

$E(x, y) + \lambda V(y)$



In [ ]:
def denoising_1D_TV(Y, lamda):
    
    N = len(Y)
    X = np.zeros(N)

    k, k0, kz, kf = 0, 0, 0, 0
    vmin = Y[0] - lamda
    vmax = Y[0] + lamda
    umin = lamda
    umax = -lamda

    while k < N:
        
        if k == N - 1:
            X[k] = vmin + umin
            break
        
        if Y[k + 1] < vmin - lamda - umin:
            for i in range(k0, kf + 1):
                X[i] = vmin
            k, k0, kz, kf = kf + 1, kf + 1, kf + 1, kf + 1
            vmin = Y[k]
            vmax = Y[k] + 2 * lamda
            umin = lamda
            umax = -lamda
            
        elif Y[k + 1] > vmax + lamda - umax:
            for i in range(k0, kz + 1):
                X[i] = vmax
            k, k0, kz, kf = kz + 1, kz + 1, kz + 1, kz + 1
            vmin = Y[k] - 2 * lamda
            vmax = Y[k]
            umin = lamda
            umax = -lamda
            
        else:
            k += 1
            umin = umin + Y[k] - vmin
            umax = umax + Y[k] - vmax
            if umin >= lamda:
                vmin = vmin + (umin - lamda) * 1.0 / (k - k0 + 1)
                umin = lamda
                kf = k
            if umax <= -lamda:
                vmax = vmax + (umax + lamda) * 1.0 / (k - k0 + 1)
                umax = -lamda
                kz = k
                
        if k == N - 1:
            if umin < 0:
                for i in range(k0, kf + 1):
                    X[i] = vmin
                k, k0, kf = kf + 1, kf + 1, kf + 1
                vmin = Y[k]
                umin = lamda
                umax = Y[k] + lamda - vmax
                
            elif umax > 0:
                for i in range(k0, kz + 1):
                    X[i] = vmax
                k, k0, kz = kz + 1, kz + 1, kz + 1
                vmax = Y[k]
                umax = -lamda
                umin = Y[k] - lamda - vmin
                
            else:
                for i in range(k0, N):
                    X[i] = vmin + umin * 1.0 / (k - k0 + 1)
                break

    return X



Experimental lambda values, adding $\alpha$:

$E(x, y) + \lambda^\alpha V(y)$

if $\lambda=10$

$E(x, y) + 10^\alpha V(y)$

with:

$\alpha = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]$

In [ ]:
def plot_tvcurves(img, x1, x2, y1, y2, alphas):
    fig, ax = plt.subplots(3, 1, figsize=(20, 18), dpi=96)
    ax[0].imshow(img, cmap='gray', aspect='auto')
    ax[1].scatter(x1, y1, alpha=1, marker='*', s=25, c='red')
    ax[2].scatter(x2, y2, alpha=0.5, s=25, c='green')
    f1 = interp1d(x1, y1, kind='linear', fill_value="extrapolate")
    f2 = interp1d(x2, y2, kind='linear', fill_value="extrapolate")
    xnew = np.linspace(0, img.shape[1], num=128)
    ax[1].plot(xnew, f1(xnew), linewidth=1, label='max1', c='red')
    ax[2].plot(xnew, f2(xnew), linewidth=1, label='max2', c='green')
    for idx, alpha in enumerate(alphas):
        lamda = 10**alpha
        X1 = denoising_1D_TV(f1(xnew), lamda)
        ax[1].plot(X1, '--', linewidth=1, label=f'alphaMax1: {alpha:0.4f}')
        X2 = denoising_1D_TV(f2(xnew), lamda)
        ax[2].plot(X2, '--', linewidth=1, label=f'alphaMax2: {alpha:0.4f}')
    
    # ax[0].set_xlim([0, img.shape[1]])
    ax[0].set_ylim([60, 100])
    ax[1].set_ylim([75, 85])
    ax[2].set_ylim([75, 85])
    ax[1].invert_yaxis()
    ax[2].invert_yaxis()
    
    ax[0].set_title('Original image', fontsize=20, fontweight='bold')
    ax[0].set_ylabel('Image Height\n (Pixels)', fontsize=14, fontweight='bold')
    ax[0].set_xlabel('Image Width\n (Pixels)', fontsize=14, fontweight='bold')
    ax[0].set_ylabel('Image Height\n (Pixels)', fontsize=14, fontweight='bold')
    ax[1].set_xlabel('Image Width\n (Pixels)', fontsize=14, fontweight='bold')
    ax[1].set_ylabel('Image Height\n (Pixels)', fontsize=14, fontweight='bold')
    ax[2].set_xlabel('Image Width\n (Pixels)', fontsize=14, fontweight='bold')
    ax[2].set_ylabel('Image Height\n (Pixels)', fontsize=14, fontweight='bold')
    ax[1].legend(loc='best', fontsize=8)
    ax[2].legend(loc='best', fontsize=8)
    plt.show()

alphas = np.linspace(1e-2,2,10)
plot_tvcurves(mul, x1, x2, y1, y2, alphas)

In [ ]:
from skimage.restoration import denoise_tv_chambolle


def tv_denoising(img, alphas):
    fig, ax = plt.subplots(len(alphas)+1, 1, figsize=(20, 10*len(alphas)), dpi=96)
    x1, x2, y1, y2 = get_max1_max2(mul)
    f1 = interp1d(x1, y1, kind='linear', fill_value="extrapolate")
    f2 = interp1d(x2, y2, kind='linear', fill_value="extrapolate")
    xnew = np.linspace(0, mul.shape[1], num=500)
    ax[0].imshow(mul, cmap='gray', aspect='auto')
    ax[0].plot(xnew, f1(xnew), linewidth=1, c='green', label=f'Max1')
    ax[0].plot(xnew, f2(xnew), linewidth=1, c='blue', label=f'Max2')
    ax[0].scatter(x1, y1, alpha=1, marker='*', s=25, c='green')
    ax[0].scatter(x2, y2, alpha=0.5, s=25, c='blue')
    ax[0].set_xlim([0, mul.shape[1]])
    ax[0].set_ylim([70, 90])
    ax[0].legend(loc='best')
    ax[0].set_ylabel('Image Height\n (Pixels)', fontsize=14, fontweight='bold')
    ax[0].set_xlabel('Image Width\n (Pixels)', fontsize=14, fontweight='bold')
    ax[0].set_title(f'Original Image', fontsize=14, fontweight='bold')
    for idx, w in enumerate(alphas):
        out = denoise_tv_chambolle(img, weight=w)
        print(out.max())
        mul_tv = np.multiply(out, msk).astype('float')
        x1, x2, y1, y2 = get_max1_max2(mul_tv)
        f1 = interp1d(x1, y1, kind='linear', fill_value="extrapolate")
        f2 = interp1d(x2, y2, kind='linear', fill_value="extrapolate")
        xnew = np.linspace(0, mul_tv.shape[1], num=500)
        ax[idx+1].imshow(mul_tv, cmap='gray', aspect='auto')
        ax[idx+1].plot(xnew, f1(xnew), linewidth=1, c='green', label=f'Max1')
        ax[idx+1].plot(xnew, f2(xnew), linewidth=1, c='blue', label=f'Max2')
        ax[idx+1].scatter(x1, y1, alpha=1, marker='*', s=25, c='green')
        ax[idx+1].scatter(x2, y2, alpha=0.5, s=25, c='blue')
        ax[idx+1].invert_yaxis()
        ax[idx+1].set_xlim([0, mul_tv.shape[1]])
        ax[idx+1].set_ylim([70, 90])
        ax[idx+1].set_ylabel('Image Height\n (Pixels)', fontsize=14, fontweight='bold')
        ax[idx+1].set_xlabel('Image Width\n (Pixels)', fontsize=14, fontweight='bold')
        ax[idx+1].legend(loc='best')
        ax[idx+1].set_title(f'Alpha: {w}', fontsize=14, fontweight='bold')
    fig.suptitle('TV Denoising', fontsize=22, fontweight='bold')
    plt.show()
    
alphas = np.linspace(5e-3,3e-1,5)
print(alphas)
tv_denoising(img, alphas)


In [ ]:
import skimage.filters as flt
import numpy as np
import scipy.ndimage.filters as flt
import warnings

def anisodiff(img, niter=1, kappa=50, gamma=0.1, step=(1.,1.), sigma=0, option=1, ploton=True):
	"""
	Anisotropic diffusion.

	Usage:
	imgout = anisodiff(im, niter, kappa, gamma, option)

	Arguments:
	        img    - input image
	        niter  - number of iterations
	        kappa  - conduction coefficient 20-100 ?
	        gamma  - max value of .25 for stability
	        step   - tuple, the distance between adjacent pixels in (y,x)
	        option - 1 Perona Malik diffusion equation No 1
	                 2 Perona Malik diffusion equation No 2
	        ploton - if True, the image will be plotted on every iteration

	Returns:
	        imgout   - diffused image.

	kappa controls conduction as a function of gradient.  If kappa is low
	small intensity gradients are able to block conduction and hence diffusion
	across step edges.  A large value reduces the influence of intensity
	gradients on conduction.

	gamma controls speed of diffusion (you usually want it at a maximum of
	0.25)

	step is used to scale the gradients in case the spacing between adjacent
	pixels differs in the x and y axes

	Diffusion equation 1 favours high contrast edges over low contrast ones.
	Diffusion equation 2 favours wide regions over smaller ones.

	Reference: 
	P. Perona and J. Malik. 
	Scale-space and edge detection using ansotropic diffusion.
	IEEE Transactions on Pattern Analysis and Machine Intelligence, 
	12(7):629-639, July 1990.

	Original MATLAB code by Peter Kovesi  
	School of Computer Science & Software Engineering
	The University of Western Australia
	pk @ csse uwa edu au
	<http://www.csse.uwa.edu.au>

	Translated to Python and optimised by Alistair Muldal
	Department of Pharmacology
	University of Oxford
	<alistair.muldal@pharm.ox.ac.uk>

	June 2000  original version.       
	March 2002 corrected diffusion eqn No 2.
	July 2012 translated to Python
	"""

	# ...you could always diffuse each color channel independently if you
	# really want
	if img.ndim == 3:
		warnings.warn("Only grayscale images allowed, converting to 2D matrix")
		img = img.mean(2)

	# initialize output array
	img = img.astype('float32')
	imgout = img.copy()

	# initialize some internal variables
	deltaS = np.zeros_like(imgout)
	deltaE = deltaS.copy()
	NS = deltaS.copy()
	EW = deltaS.copy()
	gS = np.ones_like(imgout)
	gE = gS.copy()

	# create the plot figure, if requested
	if ploton:
		import pylab as pl
		from time import sleep

		fig = pl.figure(figsize=(10,5.5),num="Anisotropic diffusion")
		ax1,ax2 = fig.add_subplot(1,2,1),fig.add_subplot(1,2,2)

		ax1.imshow(img,interpolation='nearest', cmap='gray')
		ih = ax2.imshow(imgout,interpolation='nearest',animated=True, cmap='gray')
		ax1.set_title("Original image")
		ax2.set_title("Iteration 0")

		fig.canvas.draw()

	for ii in np.arange(1,niter):

		# calculate the diffs
		deltaS[:-1,: ] = np.diff(imgout,axis=0)
		deltaE[: ,:-1] = np.diff(imgout,axis=1)

		if 0<sigma:
			deltaSf=flt.gaussian_filter(deltaS,sigma);
			deltaEf=flt.gaussian_filter(deltaE,sigma);
		else: 
			deltaSf=deltaS;
			deltaEf=deltaE;
			
		# conduction gradients (only need to compute one per dim!)
		if option == 1:
			gS = np.exp(-(deltaSf/kappa)**2.)/step[0]
			gE = np.exp(-(deltaEf/kappa)**2.)/step[1]
		elif option == 2:
			gS = 1./(1.+(deltaSf/kappa)**2.)/step[0]
			gE = 1./(1.+(deltaEf/kappa)**2.)/step[1]

		# update matrices
		E = gE*deltaE
		S = gS*deltaS

		# subtract a copy that has been shifted 'North/West' by one
		# pixel. don't as questions. just do it. trust me.
		NS[:] = S
		EW[:] = E
		NS[1:,:] -= S[:-1,:]
		EW[:,1:] -= E[:,:-1]

		# update the image
		imgout += gamma*(NS+EW)

		if ploton:
			iterstring = "Difussion"
			ih.set_data(imgout)
			ax2.set_title(iterstring)
			fig.canvas.draw()
			# sleep(0.01)

	return imgout

def anisotropic_filter(img):
	iterations = [2, 4, 6, 8, 10]
	fig, ax = plt.subplots(len(iterations)+1, 1, figsize=(20, 10*len(iterations)), dpi=96)
	
	x1, x2, y1, y2 = get_max1_max2(mul)
	f1 = interp1d(x1, y1, kind='linear', fill_value="extrapolate")
	f2 = interp1d(x2, y2, kind='linear', fill_value="extrapolate")
	xnew = np.linspace(0, mul.shape[1], num=500)
	ax[0].imshow(mul, cmap='gray', aspect='auto')
	ax[0].set_xlim([0, mul.shape[1]])
	ax[0].plot(xnew, f1(xnew), linewidth=1, c='green', label=f'Max1')
	ax[0].plot(xnew, f2(xnew), linewidth=1, c='blue', label=f'Max2')
	ax[0].scatter(x1, y1, alpha=1, marker='*', s=25, c='green')
	ax[0].scatter(x2, y2, alpha=0.5, s=25, c='blue')
	ax[0].invert_yaxis()
	ax[0].set_ylim([70, 90])
	ax[0].set_title('Original Image', fontsize=14, fontweight='bold')
	ax[0].legend(loc='best')
	for idx, w in enumerate(iterations):
		out = anisodiff(img, niter=w, kappa=50, gamma=0.1)
		mul_aniso = np.multiply(out, msk).astype('float')
		x1, x2, y, y2 = get_max1_max2(mul_aniso)
		f1 = interp1d(x1, y1, kind='linear', fill_value="extrapolate")
		f2 = interp1d(x2, y2, kind='linear', fill_value="extrapolate")
		xnew = np.linspace(0, mul_aniso.shape[1], num=500)
		ax[idx+1].imshow(mul_aniso, cmap='gray', aspect='auto')
		ax[idx+1].plot(xnew, f1(xnew), linewidth=1, c='green', label=f'Max1')
		ax[idx+1].plot(xnew, f2(xnew), linewidth=1, c='blue', label=f'Max2')
		ax[idx+1].scatter(x1, y1, alpha=1, marker='*', s=25, c='green')
		ax[idx+1].scatter(x2, y2, alpha=0.5, s=25, c='blue')
		ax[idx+1].invert_yaxis()
		ax[idx+1].set_xlim([0, mul_aniso.shape[1]])
		ax[idx+1].set_ylim([70, 90])
		ax[idx+1].set_ylabel('Image Height\n (Pixels)', fontsize=14, fontweight='bold')
		ax[idx+1].set_xlabel('Image Width\n (Pixels)', fontsize=14, fontweight='bold')
		ax[idx+1].legend(loc='best')
		ax[idx+1].set_title(f'Difussion iterations: {w}, gamma: {0.1}', fontsize=14, fontweight='bold')
		# ax[2].imshow(out, cmap='gray')
	fig.suptitle('Anisotropic difussion', fontsize=22, fontweight='bold')
	plt.show()

anisotropic_filter(img)


In [ ]:
from scipy.ndimage import gaussian_filter

def gaussian_op(img):
	sigmas = [0.5, 1, 1.5, 2, 2.5]
	fig, ax = plt.subplots(len(sigmas)+1, 1, figsize=(20, 10*len(sigmas)), dpi=96)
	
	x1, x2, y1, y2 = get_max1_max2(mul)
	f1 = interp1d(x1, y1, kind='linear', fill_value="extrapolate")
	f2 = interp1d(x2, y2, kind='linear', fill_value="extrapolate")
	xnew = np.linspace(0, mul.shape[1], num=500)
	ax[0].imshow(mul, cmap='gray', aspect='auto')
	ax[0].set_xlim([0, mul.shape[1]])
	ax[0].plot(xnew, f1(xnew), linewidth=1, c='green', label=f'Max1')
	ax[0].plot(xnew, f2(xnew), linewidth=1, c='blue', label=f'Max2')
	ax[0].scatter(x1, y1, alpha=1, marker='*', s=25, c='green')
	ax[0].scatter(x2, y2, alpha=0.5, s=25, c='blue')
	ax[0].invert_yaxis()
	ax[0].set_ylim([70, 90])
	ax[0].set_title('Original Image', fontsize=14, fontweight='bold')
	ax[0].legend(loc='best')
	for idx, w in enumerate(sigmas):
		out = gaussian_filter(img, sigma=1)
		mul_aniso = np.multiply(out, msk).astype('float')
		x1, x2, y, y2 = get_max1_max2(mul_aniso)
		f1 = interp1d(x1, y1, kind='linear', fill_value="extrapolate")
		f2 = interp1d(x2, y2, kind='linear', fill_value="extrapolate")
		xnew = np.linspace(0, mul_aniso.shape[1], num=500)
		ax[idx+1].imshow(mul_aniso, cmap='gray', aspect='auto')
		ax[idx+1].plot(xnew, f1(xnew), linewidth=1, c='green', label=f'Max1')
		ax[idx+1].plot(xnew, f2(xnew), linewidth=1, c='blue', label=f'Max2')
		ax[idx+1].scatter(x1, y1, alpha=1, marker='*', s=25, c='green')
		ax[idx+1].scatter(x2, y2, alpha=0.5, s=25, c='blue')
		ax[idx+1].invert_yaxis()
		ax[idx+1].set_xlim([0, mul_aniso.shape[1]])
		ax[idx+1].set_ylim([70, 90])
		ax[idx+1].set_ylabel('Image Height\n (Pixels)', fontsize=14, fontweight='bold')
		ax[idx+1].set_xlabel('Image Width\n (Pixels)', fontsize=14, fontweight='bold')
		ax[idx+1].legend(loc='best')
		ax[idx+1].set_title(f'Gaussian filter Sigma: {w}', fontsize=14, fontweight='bold')
		# ax[2].imshow(out, cmap='gray')
	fig.suptitle('Gaussian filter', fontsize=22, fontweight='bold')
	plt.show()

gaussian_op(img)
fig = plt.figure(figsize=(10,5.5))
plt.gray()  # show the filtered result in grayscale
ax1 = fig.add_subplot(121)  # left side
ax2 = fig.add_subplot(122)  # right side
result = gaussian_filter(img, sigma=2.5)
ax1.imshow(img)
ax2.imshow(result)
plt.show()